In [6]:
import cv2
import mediapipe as mp
import time
import mediapipe.framework.formats.landmark_pb2 as landmark_pb2

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
i=0
landmarks1 = landmark_pb2.NormalizedLandmarkList()


#get a class name otherwise assign classname as working
# class_name = input("Enter a Pose for class column")
# if class_name == '':
#     class_name = "working"

#adds a countdown to get user in position for pose collection
t=2
print("running pose collection in \n \tpress Esc to close")
while t > 0:
        print(t)
        t -= 1
        time.sleep(1)

# For webcam input:
sets = 0
cap = cv2.VideoCapture(0)
predicted = ''
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      predicted = 'empty'
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    cv2.putText(image, predicted, (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, cv2.LINE_AA)

#     time.sleep(1)
    
    # Draw the pose annotation on the image.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


#     text_flipped = cv2.flip(image, 1)
#     image = cv2.flip(image, 1)
    mp_drawing.draw_landmarks(
            image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
#     image = cv2.flip(image, 1)
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    cv2.setWindowProperty("MediaPipe Pose", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    i+=1
    if (results.pose_landmarks is not None):
        if i >=20:
#         write_dict_list_to_csv(results.pose_landmarks.landmark, class_name, 'landmarks.csv')
            predicted = predict(results.pose_landmarks.landmark)
#             print(results.pose_landmarks)
            sets+=1
            print(f"sets collected {sets}")
            i=0
#     print(results.pose_landmarks)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyWindow('MediaPipe Pose')

running pose collection in 
 	press Esc to close
2
1
1/1 [==============================] - 0s 293ms/step
sets collected 1
1/1 [==============================] - 0s 35ms/step
sets collected 2
1/1 [==============================] - 0s 35ms/step
sets collected 3
1/1 [==============================] - 0s 25ms/step
sets collected 4
1/1 [==============================] - 0s 31ms/step
sets collected 5
1/1 [==============================] - 0s 23ms/step
sets collected 6
1/1 [==============================] - 0s 20ms/step
sets collected 7
1/1 [==============================] - 0s 25ms/step
sets collected 8
1/1 [==============================] - 0s 32ms/step
sets collected 9
1/1 [==============================] - 0s 31ms/step
sets collected 10
1/1 [==============================] - 0s 27ms/step
sets collected 11
1/1 [==============================] - 0s 36ms/step
sets collected 12
1/1 [==============================] - 0s 29ms/step
sets collected 13
1/1 [==============================] - 0s 34m

In [21]:
cap.release()
cv2.destroyWindow('MediaPipe Pose')

In [1]:
import pandas as pd

def convert_landmarks_to_dataframe(landmarks):
    # Check if the landmarks list is empty or contains None values
    if not landmarks or None in landmarks:
        return pd.DataFrame()
    
    # Create a dictionary with the values of each NormalizedLandmark object in the list
    row = {}
    for i in range(len(landmarks)):
        normalized_landmark = landmarks[i]
        row[f"x{i+1}"] = normalized_landmark.x
        row[f"y{i+1}"] = normalized_landmark.y
        row[f"z{i+1}"] = normalized_landmark.z
        row[f"visibility{i+1}"] = normalized_landmark.visibility
    
    # Create a Pandas dataframe from the dictionary
    df = pd.DataFrame([row])
    return df

In [64]:
convert_landmarks_to_dataframe(results.pose_landmarks.landmark)

,x1,y1,z1,visibility1,x2,y2,z2,visibility2,x3,y3,...,z31,visibility31,x32,y32,z32,visibility32,x33,y33,z33,visibility33
0,0.691393,0.918563,-1.585427,0.99655,0.723314,0.857939,-1.53929,0.994406,0.741288,0.861298,...,0.366948,0.000131,0.63705,3.272864,-0.286246,0.000288,0.475673,3.228985,-0.342598,0.00027


In [7]:
import csv
import pandas as pd

def write_dict_list_to_csv(landmark_list):
  # Open the CSV file for writing in append mode
  
    fieldnames = []
    for i in range(len(landmark_list)):
      fieldnames.extend([f"x{i+1}", f"y{i+1}", f"z{i+1}", f"visibility{i+1}"])
    # Add the class name as the last field
    fieldnames.append('class')
    

    # Create a dictionary with the values of each NormalizedLandmark object in the list
    row = {}
    for i in range(len(landmark_list)):
      normalized_landmark = landmark_list[i]
      row[f"x{i+1}"] = normalized_landmark.x
      row[f"y{i+1}"] = normalized_landmark.y
      row[f"z{i+1}"] = normalized_landmark.z
      row[f"visibility{i+1}"] = normalized_landmark.visibility
    # Add the class name to the row
    # Write the dictionary as a row in the CSV file

  
  
    data = row
    new = pd.DataFrame.from_dict(data, orient ='index')
    return new

In [6]:
x = write_dict_list_to_csv(results.pose_landmarks.landmark)

In [7]:
x.drop('class').values

array([[0.6344282031059265],
       [0.710940420627594],
       [-1.3856899738311768],
       [0.9999334812164307],
       [0.6581674814224243],
       [0.6617523431777954],
       [-1.3716752529144287],
       [0.999833345413208],
       [0.6686854958534241],
       [0.6619142293930054],
       [-1.3711966276168823],
       [0.9998397827148438],
       [0.6776990294456482],
       [0.661994218826294],
       [-1.3712471723556519],
       [0.999764621257782],
       [0.6184664964675903],
       [0.6534399390220642],
       [-1.3940351009368896],
       [0.99989253282547],
       [0.60341477394104],
       [0.6478306651115417],
       [-1.393911361694336],
       [0.9998992085456848],
       [0.5890911221504211],
       [0.6420803070068359],
       [-1.3939545154571533],
       [0.9998447299003601],
       [0.6741799116134644],
       [0.6650322079658508],
       [-1.1173044443130493],
       [0.9998070597648621],
       [0.5490282773971558],
       [0.6350849866867065],
       [-1.2176

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

# Load the model
model = load_model('model1.h5')

2023-01-04 16:08:03.275992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 16:08:03.989341: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 16:08:03.989370: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-04 16:08:05.821649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [55]:
x1 = pd.DataFrame(x.drop('class').iloc[0:].values.reshape(1, -1))

In [83]:
x1.iloc[:1]

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,0.634428,0.71094,-1.38569,0.999933,0.658167,0.661752,-1.371675,0.999833,0.668685,0.661914,...,0.273447,0.000009,0.576066,2.334294,0.006472,0.000063,0.431789,2.297956,-0.143948,0.000024


In [84]:
model.predict(x2)

1/1 [==============================] - 0s 85ms/step


array([[4.798374e-08, 9.997719e-01, 6.379073e-04]], dtype=float32)

In [70]:
print(x1.values.dtype)

object


In [78]:
x2 = x1.apply(pd.to_numeric, errors='coerce', downcast='float')
x2.dtypes

0      float32
1      float32
2      float32
3      float32
4      float32
        ...   
127    float32
128    float32
129    float32
130    float32
131    float32
Length: 132, dtype: object

In [86]:
import numpy as np
# predictions = model.predict(df.iloc[:1].drop('class', axis=1))
index = 1000

# Create a dictionary that maps the class numbers to the class labels
class_labels = {0: 'away', 1: 'phone', 2: 'working'}

# Make a prediction
prediction = model.predict(x2)

# Find the index of the highest probability
class_index = np.argmax(prediction)

# Look up the class label in the dictionary
class_label = class_labels[class_index]

# Print the class label
print("Class label:", class_label)

# print("actual: ", df.iloc[index-1:index]['class'])

1/1 [==============================] - 0s 44ms/step
Class label: phone


In [3]:
import numpy as np

def predict(landmark):
    x = convert_landmarks_to_dataframe(landmark)
#     x = pd.DataFrame(x.iloc[0:].values.reshape(1, -1))
    x = x.apply(pd.to_numeric, errors='coerce', downcast='float')
    class_labels = {0: 'away', 1: 'phone', 2: 'working'}

    # Make a prediction
    prediction = model.predict(x)

    # Find the index of the highest probability
    class_index = np.argmax(prediction)

    # Look up the class label in the dictionary
    class_label = class_labels[class_index]

    # Print the class label
#     print("Class label:", class_label)
    return class_label

In [4]:
predict(results.pose_landmarks.landmark)

NameError: name 'results' is not defined

In [157]:
import cv2

# Start the webcam and get a video capture object
capture = cv2.VideoCapture(0)

while True:
  # Capture a frame from the webcam
  _, frame = capture.read()

  # Flip the frame horizontally
  frame = cv2.flip(frame, 1)

  # Draw the text on the frame
  cv2.putText(frame, "Hello, World!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)

  # Flip the text horizontally
#   frame = cv2.flip(frame, 1)

  # Display the frame
  cv2.imshow("Flipped Text", frame)

  # Wait for a key press to exit the loop
  key = cv2.waitKey(1)
  if key == ord("q"):
    break

# Release the video capture object and close all windows
capture.release()
cv2.destroyAllWindows()


In [8]:
x = {'landmarks': [{'x': 0.5454695820808411, 'y': 0.6748321652412415, 'z': -0.7199423313140869, 'visibility': 0.99909907579422}, {'x': 0.5473164916038513, 'y': 0.6220959424972534, 'z': -0.6980997920036316, 'visibility': 0.9982926845550537}, {'x': 0.5515623688697815, 'y': 0.6206902265548706, 'z': -0.6980924606323242, 'visibility': 0.997909665107727}, {'x': 0.5557631254196167, 'y': 0.6185289025306702, 'z': -0.6980994343757629, 'visibility': 0.9973894357681274}, {'x': 0.5257865190505981, 'y': 0.6207806468009949, 'z': -0.7406908869743347, 'visibility': 0.9990617036819458}, {'x': 0.5132017731666565, 'y': 0.6181095838546753, 'z': -0.7406903505325317, 'visibility': 0.9991831183433533}, {'x': 0.500263512134552, 'y': 0.614709198474884, 'z': -0.7410531640052795, 'visibility': 0.9991683959960938}, {'x': 0.5398885607719421, 'y': 0.6278904676437378, 'z': -0.5105078816413879, 'visibility': 0.9978160262107849}, {'x': 0.4619547724723816, 'y': 0.6226251125335693, 'z': -0.6971204876899719, 'visibility': 0.9996634721755981}, {'x': 0.5430112481117249, 'y': 0.7263548374176025, 'z': -0.6406038999557495, 'visibility': 0.9987558722496033}, {'x': 0.5178163051605225, 'y': 0.7259326577186584, 'z': -0.6942938566207886, 'visibility': 0.9995691180229187}, {'x': 0.5578140020370483, 'y': 0.8343484401702881, 'z': -0.3178628087043762, 'visibility': 0.9890120029449463}, {'x': 0.3234836459159851, 'y': 0.8491232395172119, 'z': -0.5962515473365784, 'visibility': 0.995445191860199}, {'x': 0.6168203949928284, 'y': 1.0795156955718994, 'z': -0.09192551672458649, 'visibility': 0.19562563300132751}, {'x': 0.25853151082992554, 'y': 1.2304266691207886, 'z': -0.5265306830406189, 'visibility': 0.6719987988471985}, {'x': 0.6316277980804443, 'y': 1.294083833694458, 'z': -0.05329129844903946, 'visibility': 0.11209479719400406}, {'x': 0.3096488118171692, 'y': 1.4799680709838867, 'z': -0.5408157110214233, 'visibility': 0.21171773970127106}, {'x': 0.6532174348831177, 'y': 1.3665655851364136, 'z': -0.05721629783511162, 'visibility': 0.12234904617071152}, {'x': 0.3093355596065521, 'y': 1.573687195777893, 'z': -0.5664710402488708, 'visibility': 0.198204904794693}, {'x': 0.6420430541038513, 'y': 1.3811053037643433, 'z': -0.08566587418317795, 'visibility': 0.15820267796516418}, {'x': 0.33128952980041504, 'y': 1.556300163269043, 'z': -0.6115931272506714, 'visibility': 0.2556341886520386}, {'x': 0.6234787106513977, 'y': 1.3598535060882568, 'z': -0.06904463469982147, 'visibility': 0.16368260979652405}, {'x': 0.3348403573036194, 'y': 1.517934799194336, 'z': -0.5585611462593079, 'visibility': 0.24491675198078156}, {'x': 0.5063937902450562, 'y': 1.4139409065246582, 'z': 0.035775117576122284, 'visibility': 0.005176164675503969}, {'x': 0.34810250997543335, 'y': 1.4274227619171143, 'z': -0.034507159143686295, 'visibility': 0.006485120393335819}, {'x': 0.5042237043380737, 'y': 1.8846806287765503, 'z': 0.11370231211185455, 'visibility': 0.01315288431942463}, {'x': 0.36905908584594727, 'y': 1.9186561107635498, 'z': 0.015820426866412163, 'visibility': 0.01378056313842535}, {'x': 0.4914855360984802, 'y': 2.3103840351104736, 'z': 0.4601244330406189, 'visibility': 0.001576005364768207}, {'x': 0.3653518557548523, 'y': 2.3146400451660156, 'z': 0.2707299590110779, 'visibility': 0.0012553343549370766}, {'x': 0.48404645919799805, 'y': 2.3743553161621094, 'z': 0.4768470227718353, 'visibility': 0.0010925508104264736}, {'x': 0.35052287578582764, 'y': 2.3798959255218506, 'z': 0.2834347188472748, 'visibility': 0.0007594606140628457}, {'x': 0.5156856775283813, 'y': 2.4439892768859863, 'z': 0.23236814141273499, 'visibility': 0.0035022960510104895}, {'x': 0.4108780026435852, 'y': 2.4339187145233154, 'z': -0.024153608828783035, 'visibility': 0.0028416551649570465}]}

In [7]:
org = convert_landmarks_to_dataframe(results.pose_landmarks.landmark)
print(org.shape)
print(type(org))
org

(1, 132)
<class 'pandas.core.frame.DataFrame'>


,x1,y1,z1,visibility1,x2,y2,z2,visibility2,x3,y3,...,z31,visibility31,x32,y32,z32,visibility32,x33,y33,z33,visibility33
0,0.455304,0.472088,-1.201609,0.998883,0.479849,0.423676,-1.170428,0.997096,0.492329,0.424069,...,0.012189,0.000389,0.548108,2.384305,-0.167,0.00068,0.407218,2.368752,-0.499174,0.00083


In [8]:
# Extract the list of landmarks from the dictionary
x = {'landmarks': [{'x': 0.4686046242713928, 'y': 0.5030984282493591, 'z': -1.0094232559204102, 'visibility': 0.9999529719352722}, {'x': 0.48775234818458557, 'y': 0.4446941018104553, 'z': -0.9871283769607544, 'visibility': 0.9998947978019714}, {'x': 0.49852919578552246, 'y': 0.4435579180717468, 'z': -0.986343502998352, 'visibility': 0.9999161958694458}, {'x': 0.5082883238792419, 'y': 0.4432961344718933, 'z': -0.986343502998352, 'visibility': 0.9998880624771118}, {'x': 0.4527718126773834, 'y': 0.4426792562007904, 'z': -0.9944409132003784, 'visibility': 0.9999018311500549}, {'x': 0.4411553740501404, 'y': 0.44038116931915283, 'z': -0.9945420026779175, 'visibility': 0.9999213814735413}, {'x': 0.4302472174167633, 'y': 0.4392113983631134, 'z': -0.9945403337478638, 'visibility': 0.9999013543128967}, {'x': 0.5224308371543884, 'y': 0.451479971408844, 'z': -0.7177700996398926, 'visibility': 0.9999082088470459}, {'x': 0.4138098657131195, 'y': 0.4417473375797272, 'z': -0.7355228662490845, 'visibility': 0.9999228119850159}, {'x': 0.48767852783203125, 'y': 0.559472382068634, 'z': -0.8971085548400879, 'visibility': 0.9999293088912964}, {'x': 0.44671162962913513, 'y': 0.555407702922821, 'z': -0.903830349445343, 'visibility': 0.9999430775642395}, {'x': 0.616666316986084, 'y': 0.7058625817298889, 'z': -0.46716952323913574, 'visibility': 0.9994540810585022}, {'x': 0.3250430226325989, 'y': 0.670527458190918, 'z': -0.509357750415802, 'visibility': 0.9993830323219299}, {'x': 0.6688510179519653, 'y': 1.0034350156784058, 'z': -0.3775273561477661, 'visibility': 0.5262294411659241}, {'x': 0.23810502886772156, 'y': 0.9780781269073486, 'z': -0.4023509621620178, 'visibility': 0.5915988683700562}, {'x': 0.6724638342857361, 'y': 1.3393617868423462, 'z': -0.5976381897926331, 'visibility': 0.16969288885593414}, {'x': 0.22594305872917175, 'y': 1.260507345199585, 'z': -0.6331285834312439, 'visibility': 0.43696948885917664}, {'x': 0.6889642477035522, 'y': 1.4360429048538208, 'z': -0.662922203540802, 'visibility': 0.15097324550151825}, {'x': 0.20606979727745056, 'y': 1.3354556560516357, 'z': -0.7017050981521606, 'visibility': 0.3797420561313629}, {'x': 0.657593309879303, 'y': 1.4342248439788818, 'z': -0.7161909937858582, 'visibility': 0.22486940026283264}, {'x': 0.22511446475982666, 'y': 1.327080249786377, 'z': -0.7717355489730835, 'visibility': 0.4600752592086792}, {'x': 0.6459468603134155, 'y': 1.3913002014160156, 'z': -0.629507839679718, 'visibility': 0.2301783561706543}, {'x': 0.23753654956817627, 'y': 1.295259714126587, 'z': -0.6691960096359253, 'visibility': 0.44627952575683594}, {'x': 0.5529295206069946, 'y': 1.349688172340393, 'z': -0.02225726842880249, 'visibility': 0.0013633813941851258}, {'x': 0.3571010231971741, 'y': 1.3454691171646118, 'z': 0.02544519305229187, 'visibility': 0.0013641009572893381}, {'x': 0.545749306678772, 'y': 1.8947969675064087, 'z': -0.020583773031830788, 'visibility': 0.0021279235370457172}, {'x': 0.36439329385757446, 'y': 1.8940032720565796, 'z': -0.017163928598165512, 'visibility': 0.0006514777196571231}, {'x': 0.5402830243110657, 'y': 2.3830039501190186, 'z': 0.35081639885902405, 'visibility': 0.00015770185564178973}, {'x': 0.3682359457015991, 'y': 2.3809101581573486, 'z': 0.21661706268787384, 'visibility': 3.268474029027857e-05}, {'x': 0.5442759990692139, 'y': 2.4659066200256348, 'z': 0.3664318919181824, 'visibility': 0.0001172124466393143}, {'x': 0.3669081926345825, 'y': 2.46389102935791, 'z': 0.22608837485313416, 'visibility': 3.938096415367909e-05}, {'x': 0.5060566663742065, 'y': 2.549800395965576, 'z': -0.01518632285296917, 'visibility': 0.00016118095663841814}, {'x': 0.3937321603298187, 'y': 2.543214797973633, 'z': -0.19619040191173553, 'visibility': 0.00014634219405706972}]}
from collections import OrderedDict

landmarks = x['landmarks']

# Create an empty dataframe with the required column names
for i in range(len(landmarks)):
    column_names = [f"x{i+1}", f"y{i+1}", f"z{i+1}", f"visibility{i+1}"]
df = pd.DataFrame(columns=column_names)

# Create a dictionary with the values of each landmark
row = {}
for i in range(len(landmarks)):
    landmark = landmarks[i]
    row[f"x{i+1}"] = landmark['x']
    row[f"y{i+1}"] = landmark['y']
    row[f"z{i+1}"] = landmark['z']
    row[f"visibility{i+1}"] = landmark['visibility']

# Append the dictionary as a row to the dataframe
df = df.append(row, ignore_index=True)
print(df.shape)
print(type(df))
df

(1, 132)
<class 'pandas.core.frame.DataFrame'>


/tmp/ipykernel_10065/2540544077.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


,x33,y33,z33,visibility33,x1,y1,z1,visibility1,x2,y2,...,z30,visibility30,x31,y31,z31,visibility31,x32,y32,z32,visibility32
0,0.393732,2.543215,-0.19619,0.000146,0.468605,0.503098,-1.009423,0.999953,0.487752,0.444694,...,0.366432,0.000117,0.366908,2.463891,0.226088,0.000039,0.506057,2.5498,-0.015186,0.000161


In [34]:
x['landmarks']

[{'x': 0.5454695820808411,
  'y': 0.6748321652412415,
  'z': -0.7199423313140869,
  'visibility': 0.99909907579422},
 {'x': 0.5473164916038513,
  'y': 0.6220959424972534,
  'z': -0.6980997920036316,
  'visibility': 0.9982926845550537},
 {'x': 0.5515623688697815,
  'y': 0.6206902265548706,
  'z': -0.6980924606323242,
  'visibility': 0.997909665107727},
 {'x': 0.5557631254196167,
  'y': 0.6185289025306702,
  'z': -0.6980994343757629,
  'visibility': 0.9973894357681274},
 {'x': 0.5257865190505981,
  'y': 0.6207806468009949,
  'z': -0.7406908869743347,
  'visibility': 0.9990617036819458},
 {'x': 0.5132017731666565,
  'y': 0.6181095838546753,
  'z': -0.7406903505325317,
  'visibility': 0.9991831183433533},
 {'x': 0.500263512134552,
  'y': 0.614709198474884,
  'z': -0.7410531640052795,
  'visibility': 0.9991683959960938},
 {'x': 0.5398885607719421,
  'y': 0.6278904676437378,
  'z': -0.5105078816413879,
  'visibility': 0.9978160262107849},
 {'x': 0.4619547724723816,
  'y': 0.6226251125335693,

In [63]:
{'landmarks': [{'x': 0.46238070726394653, 'y': 0.38820183277130127, 'z': -1.109244465827942, 'visibility': 0.9997896552085876}, {'x': 0.4818848669528961, 'y': 0.3291008770465851, 'z': -1.0756317377090454, 'visibility': 0.9995062351226807}, {'x': 0.4925313889980316, 'y': 0.32855018973350525, 'z': -1.0756490230560303, 'visibility': 0.9995460510253906}, {'x': 0.5027801394462585, 'y': 0.3285892605781555, 'z': -1.0755927562713623, 'visibility': 0.999375581741333}, {'x': 0.4471910893917084, 'y': 0.32595276832580566, 'z': -1.0887584686279297, 'visibility': 0.9996391534805298}, {'x': 0.43528103828430176, 'y': 0.324033260345459, 'z': -1.0887510776519775, 'visibility': 0.9997225999832153}, {'x': 0.42416971921920776, 'y': 0.3227127194404602, 'z': -1.0894914865493774, 'visibility': 0.9996910095214844}, {'x': 0.5137720108032227, 'y': 0.336418092250824, 'z': -0.7601109743118286, 'visibility': 0.9992107152938843}, {'x': 0.4019496440887451, 'y': 0.32614338397979736, 'z': -0.815753161907196, 'visibility': 0.9997750520706177}, {'x': 0.47921663522720337, 'y': 0.44850218296051025, 'z': -0.9824153780937195, 'visibility': 0.9996408820152283}, {'x': 0.43937787413597107, 'y': 0.4407077729701996, 'z': -0.9986804127693176, 'visibility': 0.9998266696929932}, {'x': 0.5946645736694336, 'y': 0.5854211449623108, 'z': -0.4586849808692932, 'visibility': 0.9981916546821594}, {'x': 0.3093334436416626, 'y': 0.613568902015686, 'z': -0.5840080976486206, 'visibility': 0.9989511370658875}, {'x': 0.6522829532623291, 'y': 0.9297661781311035, 'z': -0.2987522482872009, 'visibility': 0.5824244618415833}, {'x': 0.25302448868751526, 'y': 0.9570732116699219, 'z': -0.46764469146728516, 'visibility': 0.7789865136146545}, {'x': 0.6932791471481323, 'y': 1.1274234056472778, 'z': -0.5232348442077637, 'visibility': 0.22142554819583893}, {'x': 0.2718499004840851, 'y': 1.277754306793213, 'z': -0.7093649506568909, 'visibility': 0.35365334153175354}, {'x': 0.7176581621170044, 'y': 1.20173978805542, 'z': -0.5777596831321716, 'visibility': 0.17842042446136475}, {'x': 0.2653007507324219, 'y': 1.3754847049713135, 'z': -0.7752432823181152, 'visibility': 0.25685039162635803}, {'x': 0.703555166721344, 'y': 1.198546051979065, 'z': -0.6364760994911194, 'visibility': 0.23485377430915833}, {'x': 0.2820360064506531, 'y': 1.3592114448547363, 'z': -0.854078471660614, 'visibility': 0.33423829078674316}, {'x': 0.6873985528945923, 'y': 1.1712055206298828, 'z': -0.5560983419418335, 'visibility': 0.23164185881614685}, {'x': 0.29082512855529785, 'y': 1.322805643081665, 'z': -0.7452731728553772, 'visibility': 0.3205030560493469}, {'x': 0.5577410459518433, 'y': 1.2687638998031616, 'z': -0.021882418543100357, 'visibility': 0.0067885639145970345}, {'x': 0.36423879861831665, 'y': 1.287376046180725, 'z': 0.02453516237437725, 'visibility': 0.007515389937907457}, {'x': 0.5681191682815552, 'y': 1.827669620513916, 'z': 0.04634478688240051, 'visibility': 0.0070983353070914745}, {'x': 0.398310124874115, 'y': 1.8507429361343384, 'z': 0.07239458709955215, 'visibility': 0.0027654815930873156}, {'x': 0.5733409523963928, 'y': 2.3235082626342773, 'z': 0.5130788087844849, 'visibility': 0.000793764425907284}, {'x': 0.4084203243255615, 'y': 2.339735507965088, 'z': 0.46640652418136597, 'visibility': 0.0004244664160069078}, {'x': 0.5763721466064453, 'y': 2.399582624435425, 'z': 0.5355064272880554, 'visibility': 0.00047835384611971676}, {'x': 0.40561017394065857, 'y': 2.415130376815796, 'z': 0.49764832854270935, 'visibility': 0.00044210461783222854}, {'x': 0.5494622588157654, 'y': 2.494516611099243, 'z': 0.13515032827854156, 'visibility': 0.0008777239127084613}, {'x': 0.44077378511428833, 'y': 2.5031063556671143, 'z': -0.011419386602938175, 'visibility': 0.0013370162341743708}]}
# pdf = pd.DataFrame()
# for i in range(1, 32):
#     for y in range(0, 3):
#         pdf[f'x{y+i}'] = [x['landmarks'][i]['x']]
#         pdf[f'y{y+i}'] = [x['landmarks'][i]['y']]
#         pdf[f'z{y+i}'] = [x['landmarks'][i]['z']]
#         pdf[f'v{y+i}'] = [x['landmarks'][i]['visibility']]
        
        
columns = []
data = []
for i in range(len(x['landmarks'])):
    columns.extend([f"x{i+1}", f"y{i+1}", f"z{i+1}", f"visibility{i+1}"])
    data.extend([x['landmarks'][i]['x'], x['landmarks'][i]['y'], x['landmarks'][i]['z'], x['landmarks'][i]['visibility']])

pdf = pd.DataFrame(data=[data], columns=columns)


In [24]:
org

,x1,y1,z1,visibility1,x2,y2,z2,visibility2,x3,y3,...,z31,visibility31,x32,y32,z32,visibility32,x33,y33,z33,visibility33
0,0.455304,0.472088,-1.201609,0.998883,0.479849,0.423676,-1.170428,0.997096,0.492329,0.424069,...,0.012189,0.000389,0.548108,2.384305,-0.167,0.00068,0.407218,2.368752,-0.499174,0.00083


In [64]:
pdf

,x1,y1,z1,visibility1,x2,y2,z2,visibility2,x3,y3,...,z31,visibility31,x32,y32,z32,visibility32,x33,y33,z33,visibility33
0,0.462381,0.388202,-1.109244,0.99979,0.481885,0.329101,-1.075632,0.999506,0.492531,0.32855,...,0.497648,0.000442,0.549462,2.494517,0.13515,0.000878,0.440774,2.503106,-0.011419,0.001337


In [65]:
class_labels = {0: 'away', 1: 'phone', 2: 'working'}

# Make a prediction
prediction = model.predict(pdf)

# Find the index of the highest probability
class_index = np.argmax(prediction)

# Look up the class label in the dictionary
class_label = class_labels[class_index]
print(class_label)

1/1 [==============================] - 0s 47ms/step
working
